In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("DelhiAQI.csv")

In [3]:
df

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
0,454.58,935.18,81.52,41.78,187.66,27.54,9.29,3.41,54.94,25.24,58.57,13.80,653
1,440.44,935.18,70.80,43.46,176.83,27.72,13.28,3.88,50.53,23.10,49.37,15.63,645
2,409.09,935.18,132.46,41.19,141.02,28.94,29.67,2.83,19.33,19.04,38.94,17.18,532
3,436.12,935.18,84.78,39.55,102.84,29.30,21.76,4.33,20.08,13.99,27.53,16.82,561
4,415.88,976.99,60.24,37.41,80.12,30.84,26.19,6.17,16.00,11.14,21.99,14.29,567
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,57.89,193.89,24.06,76.82,97.99,34.48,1.13,12.87,59.50,3.02,10.32,0.00,192
14996,56.32,200.33,21.57,68.99,87.61,28.56,1.20,11.10,67.80,3.51,12.59,0.00,192
14997,50.93,177.50,18.47,65.41,80.13,23.19,1.16,10.55,57.38,3.39,10.11,0.02,191
14998,69.22,182.72,19.67,56.50,73.91,22.19,1.00,9.67,70.40,2.48,7.02,0.00,189


In [55]:
X=df.drop(columns=['AQI'], inplace=False)
y=df['AQI']



In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_eval, y_val, y_eval=train_test_split(X_test, y_test, test_size=0.5, random_state=42)


scaler = MinMaxScaler()
X_train_np = X_train.values
X_train_scaled = scaler.fit_transform(X_train_np)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
 
X_val_np = X_val.values
X_val_scaled = scaler.transform(X_val_np)
X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=X_val.columns)
 
X_eval_np = X_eval.values
X_eval_scaled = scaler.transform(X_eval_np)
X_eval_scaled_df = pd.DataFrame(X_eval_scaled, columns=X_eval.columns)
 
scalerY = MinMaxScaler()
y_train_scaled = scalerY.fit_transform(y_train.values.reshape(-1,1))
y_val_scaled = scalerY.transform(y_val.values.reshape(-1,1))
y_eval_scaled = scalerY.transform(y_eval.values.reshape(-1,1))

In [57]:
import tensorflow as tf
from tensorflow.keras.initializers import GlorotUniform, HeUniform
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import Callback
 
class Model1:
    def __init__(Self, X_train, y_train, X_val, y_val, X_eval, y_eval):
        Self.X_train = X_train
        Self.X_val = X_val
        Self.X_eval = X_eval
        Self.y_train = y_train
        Self.y_val = y_val
        Self.y_eval = y_eval
    def train(self):
        class AccuracyCallback(Callback):
            def on_epoch_end(self, epoch, logs={}):
                print(f"Epoch {epoch + 1}, Training Accuracy: {logs['accuracy']:.4f}, Validation Accuracy: {logs['val_accuracy']:.4f}")
        accuracy_callback = AccuracyCallback()
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(32, activation='sigmoid', kernel_initializer=GlorotUniform()))
        self.model.add(tf.keras.layers.Dense(16, activation='relu', kernel_initializer=HeUniform()))
        self.model.add(tf.keras.layers.Dense(1, activation='relu', kernel_initializer=HeUniform()))
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
        self.model.fit(self.X_train, self.y_train, batch_size=16, epochs=20, validation_data=(self.X_val, self.y_val))
 
    def predict_check_mean_square_error(self, scalerY):
        predictions = self.model.predict(self.X_eval)
        inverse_transform = scalerY.inverse_transform(self.y_eval)
        predictions = self.model.predict(self.X_eval)
        inverse_pred = scalerY.inverse_transform(predictions)
        rmse = mean_squared_error(inverse_pred, inverse_transform, squared=False)
        print(f"RMSE = {rmse}")

In [58]:
m1 = Model1(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, X_eval_scaled, y_eval_scaled)
m1.train()
m1.predict_check_mean_square_error(scalerY)

Epoch 1/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1558 - mae: 0.3569 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 2/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1551 - mae: 0.3552 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 3/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1582 - mae: 0.3592 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 4/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1572 - mae: 0.3575 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 5/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1566 - mae: 0.3574 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 6/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1555 - mae: 0.3570 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 7/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1540 - mae: 0.3550 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 8/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1543 - mae: 0.3558 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 9/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [60]:
import tensorflow as tf
from tensorflow.keras.initializers import GlorotUniform, HeUniform
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import Callback
 
class Model2:
    def __init__(Self, X_train, y_train, X_val, y_val, X_eval, y_eval):
        Self.X_train = X_train
        Self.X_val = X_val
        Self.X_eval = X_eval
        Self.y_train = y_train
        Self.y_val = y_val
        Self.y_eval = y_eval
    def train(self):
        class AccuracyCallback(Callback):
            def on_epoch_end(self, epoch, logs={}):
                print(f"Epoch {epoch + 1}, Training Accuracy: {logs['accuracy']:.4f}, Validation Accuracy: {logs['val_accuracy']:.4f}")
        accuracy_callback = AccuracyCallback()
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(32, activation='sigmoid', kernel_initializer=GlorotUniform()))
        self.model.add(tf.keras.layers.Dense(8, activation='sigmoid', kernel_initializer=HeUniform()))
        self.model.add(tf.keras.layers.Dense(1, activation='relu', kernel_initializer=HeUniform()))
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
        self.model.fit(self.X_train, self.y_train, batch_size=8, epochs=20, validation_data=(self.X_val, self.y_val))
 
    def predict_check_mean_square_error(self, scalerY):
        predictions = self.model.predict(self.X_eval)
        inverse_transform = scalerY.inverse_transform(self.y_eval)
        predictions = self.model.predict(self.X_eval)
        inverse_pred = scalerY.inverse_transform(predictions)
        rmse = mean_squared_error(inverse_pred, inverse_transform, squared=False)
        print(f"RMSE = {rmse}")



m2 = Model2(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, X_eval_scaled, y_eval_scaled)
m2.train()
m2.predict_check_mean_square_error(scalerY)

Epoch 1/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0178 - mae: 0.1058 - val_loss: 0.0110 - val_mae: 0.0836
Epoch 2/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0128 - mae: 0.0899 - val_loss: 0.0106 - val_mae: 0.0819
Epoch 3/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0107 - mae: 0.0809 - val_loss: 0.0101 - val_mae: 0.0790
Epoch 4/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0111 - mae: 0.0827 - val_loss: 0.0102 - val_mae: 0.0794
Epoch 5/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0109 - mae: 0.0822 - val_loss: 0.0108 - val_mae: 0.0800
Epoch 6/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0110 - mae: 0.0820 - val_loss: 0.0101 - val_mae: 0.0792
Epoch 7/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0102 - mae: 0.0793 - val_loss: 0.0098 - val_mae: 0.0766
Epoch 8/20
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0103 - mae: 0.0795 - val_loss: 0.0096 - val_mae: 0.0758
Epoch 9/20
1125/1125 ━━━━━━━━━━━━━━━━━━━

C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [68]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

class Model3:
    def __init__(self, X_train, y_train, X_val, y_val, X_eval, y_eval, scalerY):
        self.X_train = X_train
        self.X_val = X_val
        self.X_eval = X_eval
        self.y_train = y_train
        self.y_val = y_val
        self.y_eval = y_eval
        self.scalerY = scalerY

    def build_model_1(self):
        model = tf.keras.Sequential()
        model.add(Dense(32, activation='sigmoid', kernel_initializer='glorot_uniform'))
        model.add(Dense(16, activation='relu', kernel_initializer='he_uniform', name='feature_layer_1'))
        model.add(Dense(1, activation='relu', kernel_initializer='he_uniform'))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
        return model

    def build_model_2(self):
        model = tf.keras.Sequential()
        model.add(Dense(32, activation='sigmoid', kernel_initializer='glorot_uniform'))
        model.add(Dense(8, activation='sigmoid', kernel_initializer='glorot_normal', name='feature_layer_2'))
        model.add(Dense(1, activation='relu', kernel_initializer='he_uniform'))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
        return model

    def train_models(self):
        self.model_1 = self.build_model_1()
        self.model_2 = self.build_model_2()

        # Train Model-1
        self.model_1.fit(self.X_train, self.y_train, epochs=20, batch_size=16, validation_data=(self.X_val, self.y_val))

        # Train Model-2
        self.model_2.fit(self.X_train, self.y_train, epochs=20, batch_size=8, validation_data=(self.X_val, self.y_val))

    def extract_features(self):
        # Create feature extraction models
        feature_model_1 = tf.keras.Model(inputs=self.model_1.input, outputs=self.model_1.get_layer('feature_layer_1').output)
        feature_model_2 = tf.keras.Model(inputs=self.model_2.input, outputs=self.model_2.get_layer('feature_layer_2').output)

        # Extract features from the second layers
        features_1_train = feature_model_1.predict(self.X_train)
        features_2_train = feature_model_2.predict(self.X_train)
        features_1_val = feature_model_1.predict(self.X_val)
        features_2_val = feature_model_2.predict(self.X_val)
        features_1_eval = feature_model_1.predict(self.X_eval)
        features_2_eval = feature_model_2.predict(self.X_eval)

        # Stack the features horizontally
        X_train_features = np.hstack([features_1_train, features_2_train])
        X_val_features = np.hstack([features_1_val, features_2_val])
        X_eval_features = np.hstack([features_1_eval, features_2_eval])

        return X_train_features, X_val_features, X_eval_features

    def train_svr(self, X_train_features, y_train, X_val_features, y_val):
        # Train the Support Vector Regressor
        svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
        svr.fit(X_train_features, y_train.ravel())

        # Predict on validation data
        val_predictions = svr.predict(X_val_features)
        val_rmse = mean_squared_error(y_val, val_predictions, squared=False)
        print(f"Validation RMSE = {val_rmse}")

        return svr

    def predict_check_mean_square_error(self, svr, X_eval_features):
        # Make predictions on evaluation data
        predictions = svr.predict(X_eval_features)

        # Inverse transform the predictions and true values
        y_eval_inverse = self.scalerY.inverse_transform(self.y_eval.reshape(-1, 1))
        predictions_inverse = self.scalerY.inverse_transform(predictions.reshape(-1, 1))

        # Compute RMSE
        rmse = mean_squared_error(y_eval_inverse, predictions_inverse, squared=False)
        print(f"Evaluation RMSE = {rmse}")

# Example usage (with correct preprocessing steps for scaling your data):
model3 = Model3(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, X_eval_scaled, y_eval_scaled, scalerY)
model3.train_models()
X_train_features, X_val_features, X_eval_features = model3.extract_features()
svr_model = model3.train_svr(X_train_features, y_train_scaled, X_val_features, y_val_scaled)
model3.predict_check_mean_square_error(svr_model, X_eval_features)


Epoch 1/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1541 - mae: 0.3551 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 2/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1571 - mae: 0.3578 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 3/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1553 - mae: 0.3558 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 4/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1554 - mae: 0.3562 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 5/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1556 - mae: 0.3557 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 6/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1560 - mae: 0.3568 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 7/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1548 - mae: 0.3554 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 8/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1567 - mae: 0.3577 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 9/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - lo

ValueError: The layer sequential_51 has never been called and thus has no defined input.

In [71]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

class Model4:
    def __init__(self, X_train, y_train, X_val, y_val, X_eval, y_eval, scalerY, input_dim):
        self.X_train = X_train
        self.X_val = X_val
        self.X_eval = X_eval
        self.y_train = y_train
        self.y_val = y_val
        self.y_eval = y_eval
        self.scalerY = scalerY
        self.input_dim = input_dim

    def build_model_1(self):
        model = tf.keras.Sequential()
        model.add(Dense(32, activation='sigmoid', kernel_initializer='glorot_uniform', input_shape=(self.input_dim,)))
        model.add(Dense(16, activation='relu', kernel_initializer='he_uniform', name='feature_layer_1'))
        model.add(Dense(1, activation='relu', kernel_initializer='he_uniform'))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
        return model

    def build_model_2(self):
        model = tf.keras.Sequential()
        model.add(Dense(32, activation='sigmoid', kernel_initializer='glorot_uniform', input_shape=(self.input_dim,)))
        model.add(Dense(8, activation='sigmoid', kernel_initializer='glorot_normal', name='feature_layer_2'))
        model.add(Dense(1, activation='relu', kernel_initializer='he_uniform'))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])
        return model

    def train_models(self):
        self.model_1 = self.build_model_1()
        self.model_2 = self.build_model_2()

        # Train Model-1
        self.model_1.fit(self.X_train, self.y_train, epochs=20, batch_size=16, validation_data=(self.X_val, self.y_val))

        # Train Model-2
        self.model_2.fit(self.X_train, self.y_train, epochs=20, batch_size=8, validation_data=(self.X_val, self.y_val))

    def extract_features(self):
        # Create feature extraction models
        feature_model_1 = tf.keras.Model(inputs=self.model_1.input, outputs=self.model_1.get_layer('feature_layer_1').output)
        feature_model_2 = tf.keras.Model(inputs=self.model_2.input, outputs=self.model_2.get_layer('feature_layer_2').output)

        # Extract features from the second layers
        features_1_train = feature_model_1.predict(self.X_train)
        features_2_train = feature_model_2.predict(self.X_train)
        features_1_val = feature_model_1.predict(self.X_val)
        features_2_val = feature_model_2.predict(self.X_val)
        features_1_eval = feature_model_1.predict(self.X_eval)
        features_2_eval = feature_model_2.predict(self.X_eval)

        # Stack the features horizontally
        X_train_features = np.hstack([features_1_train, features_2_train])
        X_val_features = np.hstack([features_1_val, features_2_val])
        X_eval_features = np.hstack([features_1_eval, features_2_eval])

        return X_train_features, X_val_features, X_eval_features

    def apply_pca(self, X_train_features, X_val_features, X_eval_features, n_components=10):
        # Apply PCA to reduce the dimensionality
        from sklearn.decomposition import PCA
        pca = PCA(n_components=n_components)
        X_train_pca = pca.fit_transform(X_train_features)
        X_val_pca = pca.transform(X_val_features)
        X_eval_pca = pca.transform(X_eval_features)

        return X_train_pca, X_val_pca, X_eval_pca

    def train_svr(self, X_train_pca, y_train, X_val_pca, y_val):
        # Train the Support Vector Regressor
        svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
        svr.fit(X_train_pca, y_train.ravel())

        # Predict on validation data
        val_predictions = svr.predict(X_val_pca)
        val_rmse = mean_squared_error(y_val, val_predictions, squared=False)
        print(f"Validation RMSE = {val_rmse}")

        return svr

    def predict_check_mean_square_error(self, svr, X_eval_pca):
        # Make predictions on evaluation data
        predictions = svr.predict(X_eval_pca)

        # Inverse transform the predictions and true values
        y_eval_inverse = self.scalerY.inverse_transform(self.y_eval.reshape(-1, 1))
        predictions_inverse = self.scalerY.inverse_transform(predictions.reshape(-1, 1))

        # Compute RMSE
        rmse = mean_squared_error(y_eval_inverse, predictions_inverse, squared=False)
        print(f"Evaluation RMSE = {rmse}")

model4 = Model4(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, X_eval_scaled, y_eval_scaled, scalerY,X_train.shape[1])
model4.train_models()
X_train_features, X_val_features, X_eval_features = model4.extract_features()
X_train_pca, X_val_pca, X_eval_pca = model4.apply_pca(X_train_features, X_val_features, X_eval_features, n_components=10)
svr_model = model4.train_svr(X_train_pca, y_train_scaled, X_val_pca, y_val_scaled)
model4.predict_check_mean_square_error(svr_model, X_eval_pca)


Epoch 1/20


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1547 - mae: 0.3552 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 2/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1550 - mae: 0.3561 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 3/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1552 - mae: 0.3560 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 4/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1569 - mae: 0.3588 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 5/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1572 - mae: 0.3580 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 6/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1572 - mae: 0.3576 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 7/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1555 - mae: 0.3564 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 8/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1559 - mae: 0.3570 - val_loss: 0.1583 - val_mae: 0.3600
Epoch 9/20
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1564 

ValueError: The layer sequential_55 has never been called and thus has no defined input.